<a href="https://colab.research.google.com/github/vinaybiramane/ABSA/blob/main/FT_flan_t5_aspect_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -U datasets

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch\
    torchdata --quiet

%pip install \
    transformers \
    evaluate \
    rouge_score \
    loralib \
    peft --quiet

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 50.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import ast

<a name=''></a>
###Load Dataset and LLM



In [3]:
dataset = pd.read_csv('/content/absa_terms_polarity.csv')
dataset.head()

,Unnamed: 0,sentence,term,polarity,from,to,ln
0,0,"$160 for 2 filets, 2 sides, an appetizer and d...","['filets', 'sides', 'appetizer', 'drinks']","['neutral', 'neutral', 'neutral', 'neutral']","['11', '21', '31', '45']","['17', '26', '40', '51']","[6, 5, 9, 6]"
1,1,$20 for all you can eat sushi cannot be beaten.,['sushi'],['neutral'],['24'],['29'],[5]
2,2,$20 gets you unlimited sushi of a very high qu...,"['sushi', 'sushi places', 'quality']","['positive', 'positive', 'positive']","['23', '122', '44']","['28', '134', '51']","[5, 12, 7]"
3,3,"$6 and there is much tasty food, all of it fre...",['food'],['positive'],['27'],['31'],[4]
4,4,"($200 for 2 glasses of champagne, not too expe...","['glasses of champagne', 'bottle of wine', 'af...","['negative', 'negative', 'negative']","['12', '52', '73']","['32', '66', '92']","[20, 14, 19]"


In [4]:
X, y = dataset['sentence'], [', '.join(ast.literal_eval(x)) for x in dataset['term'].tolist()]

In [5]:
X[:10]

,sentence
0,"$160 for 2 filets, 2 sides, an appetizer and d..."
1,$20 for all you can eat sushi cannot be beaten.
2,$20 gets you unlimited sushi of a very high qu...
3,"$6 and there is much tasty food, all of it fre..."
4,"($200 for 2 glasses of champagne, not too expe..."
5,(Always ask the bartender for the SEASONAL bee...
6,(and I have eaten my share) Which impresses me...
7,"(food was delivered by a busboy, not waiter) W..."
8,- for dessert we split chocolate cake and vani...
9,- the bread at the beginning is super tasty an...


In [6]:
y[:10]

['filets, sides, appetizer, drinks',
 'sushi',
 'sushi, sushi places, quality',
 'food',
 'glasses of champagne, bottle of wine, after dinner drinks',
 'SEASONAL beer, bartender',
 'serve',
 'food, busboy, waiter, cheese, pasta, water and wine glasses, wine, meal',
 'chocolate cake, vanilla gelato (with espresso), dessert',
 "bread, pizza, margarite pizza with cold prosciutto and baby arugula on top, salad, Peter's Favourite pizza with prosciutto and baby arugula"]

In [7]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size = 0.3)
print(len(train_X),len(test_X),len(train_y),len(test_y))

1413 606 1413 606


In [8]:
train_X[-20:]

,sentence
587,"I would recommend Roxy's for that, but not for..."
63,"Also, if you don't want to sit at a certain ta..."
788,My friend ordered some of their special sushi ...
1716,Veal Parmigana - Better than Patsy's!
1657,This was my frist time at Cafe St. Bart's and ...
1568,There is no excuse for such lousy service!
1870,While the $20 entree range is not overly expen...
779,My boyfriend had the New England Chowder it wa...
1648,This place is not worth the prices.
1631,This is the best sushi in new york city - hand...


In [9]:
train_y[-20:]

['food',
 'table',
 'sushi rolls',
 'Veal Parmigana',
 'food, service',
 'service',
 'food, atmosphere, entree range',
 'New England Chowder, Lobster Bisque',
 'prices',
 'sushi',
 'lox',
 'Salads, meal',
 'garlic mashed potatoes',
 'service, food, atmosphere',
 'service',
 'salads, lettuce, walnuts',
 'waitress',
 'waitress, fried mini buns with the condensed milk and the assorted fruits on beancurd',
 'lunch',
 'prix fix, three course meal, glasses of wine, course']

In [10]:
train_X.reset_index(inplace=True,drop=True)
test_X.reset_index(inplace=True,drop=True)

Load the pre-trained [FLAN-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5) and its tokenizer directly from HuggingFace. Notice that you will be using the [small version](https://huggingface.co/google/flan-t5-base) of FLAN-T5. Setting `torch_dtype=torch.bfloat16` specifies the memory type to be used by this model.

In [11]:
train_X[10]

'Service was decent, but not as smooth as I would expect from a place with these prices and reputation.'

In [12]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


It is possible to pull out the number of model parameters and find out how many of them are trainable. The following function can be used to do that, at this stage, you do not need to go into details of it.

In [13]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


<a name=''></a>
###Test the Model with Zero Shot Inferencing

Test the model with the zero shot inferencing. You can see that the model struggles compared to the baseline summary, but it does pull out some important information from the text which indicates the model can be fine-tuned to the task at hand.

In [14]:
index = 24

review = train_X[index]
summary = train_y[index]

prompt = f"""
Provide the aspects terms related to restuarant from below review separated by comma.

'{review}'

Aspect Terms:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN terms:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Provide the aspects terms related to restuarant from below review separated by comma.

'Even though the restaurant was packed, we were seated promptly and even asked for a table upstairs with no problems.'

Aspect Terms:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN terms:
table, seated

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
restaurant


In [15]:
train = pd.DataFrame(train_X)
train.columns = ['review']
train['terms'] = train_y
train.head()

,review,terms
0,Overall A oh ya even though there is waiting i...,waiting
1,The only thing I moderately enjoyed was their ...,Grilled Chicken special with Edamame Puree
2,"The crispy chicken wasn't for us, though.",crispy chicken
3,Though you will undoubtedly be seated at a tab...,"table, atomosphere, room, spot"
4,Simple comfort food and what hot and lage port...,"comfort food, portions"


In [16]:
test = pd.DataFrame(test_X)
test.columns = ['review']
test['terms'] = test_y
test.head()

,review,terms
0,"If you live in Upper Manhattan, Siam Square is...",Thia food
1,"Yet paired with such rude service, would never...",service
2,To finish off such a delightful dinner experie...,"dessert, White Chocolate Bread Pudding with Ge..."
3,Nothing fancy but really good food with pretty...,"food, price"
4,We are very particular about sushi and were bo...,"sushi, ceviche mix (special), crab dumplings, ..."


In [17]:
train.to_csv("absa_train_data.csv",index=False)
test.to_csv("absa_test_data.csv",index=False)

In [18]:
data_files = {"train": ["absa_train_data.csv"],
              "test": ["absa_test_data.csv"]}
my_iterable_dataset = load_dataset("csv", data_files=data_files, split=None)
my_iterable_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'terms'],
        num_rows: 1413
    })
    test: Dataset({
        features: ['review', 'terms'],
        num_rows: 606
    })
})

In [19]:
def tokenize_function(example):
    start_prompt = 'Provide the comma separated aspects terms related to restuarant from below review.\n\n'
    end_prompt = '\n\nAspect Terms: '
    prompt = [start_prompt + review + end_prompt for review in example['review']]
    example['prompt'] = prompt
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids.to('cuda')
    example['labels'] = tokenizer(example["terms"], padding="max_length", truncation=True, return_tensors="pt").input_ids.to('cuda')

    return example


In [20]:
# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = my_iterable_dataset.map(tokenize_function, batched=True)
tokenized_datasets.remove_columns(['review', 'terms'])

Map:   0%|          | 0/1413 [00:00<?, ? examples/s]

Map:   0%|          | 0/606 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'input_ids', 'labels'],
        num_rows: 1413
    })
    test: Dataset({
        features: ['prompt', 'input_ids', 'labels'],
        num_rows: 606
    })
})

To save some time in the lab, you will subsample the dataset:

Check the shapes of all three parts of the dataset:

<a name=''></a>
##Perform Parameter Efficient Fine-Tuning (PEFT)

Now, let's perform **Parameter Efficient Fine-Tuning (PEFT)** fine-tuning as opposed to "full fine-tuning" as you did above. PEFT is a form of instruction fine-tuning that is much more efficient than full fine-tuning - with comparable evaluation results as you will see soon.

PEFT is a generic term that includes **Low-Rank Adaptation (LoRA)** and prompt tuning (which is NOT THE SAME as prompt engineering!). In most cases, when someone says PEFT, they typically mean LoRA. LoRA, at a very high level, allows the user to fine-tune their model using fewer compute resources (in some cases, a single GPU). After fine-tuning for a specific task, use case, or tenant with LoRA, the result is that the original LLM remains unchanged and a newly-trained “LoRA adapter” emerges. This LoRA adapter is much, much smaller than the original LLM - on the order of a single-digit % of the original LLM size (MBs vs GBs).  

That said, at inference time, the LoRA adapter needs to be reunited and combined with its original LLM to serve the inference request.  The benefit, however, is that many LoRA adapters can re-use the original LLM which reduces overall memory requirements when serving multiple tasks and use cases.

<a name=''></a>
###Setup the PEFT/LoRA model for Fine-Tuning

You need to set up the PEFT/LoRA model for fine-tuning with a new layer/parameter adapter. Using PEFT/LoRA, you are freezing the underlying LLM and only training the adapter. Have a look at the LoRA configuration below. Note the rank (`r`) hyper-parameter, which defines the rank/dimension of the adapter to be trained.

In [21]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=256, # Rank
    lora_alpha=128,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

Add LoRA adapter layers/parameters to the original LLM to be trained.

In [22]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 28311552
all model parameters: 275889408
percentage of trainable model parameters: 10.26%


<a name=''></a>
###Train PEFT Adapter

Define training arguments and create `Trainer` instance.

In [23]:
output_dir = f'./peft-detect-aspect-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-4, # Higher learning rate than full fine-tuning.
    num_train_epochs=10,
    logging_steps=1,
    max_steps=20
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets['train'],
)

max_steps is given, it will override any value given in num_train_epochs


Now everything is ready to train the PEFT adapter and save the model.

In [24]:
peft_trainer.train()

peft_model_path="./peft-detect-aspects-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
1,41.250000
2,39.750000
3,36.750000
4,39.250000
5,37.000000
6,36.500000
7,37.000000
8,34.500000
9,35.250000
10,33.750000


('./peft-detect-aspects-checkpoint-local/tokenizer_config.json',
 './peft-detect-aspects-checkpoint-local/special_tokens_map.json',
 './peft-detect-aspects-checkpoint-local/spiece.model',
 './peft-detect-aspects-checkpoint-local/added_tokens.json',
 './peft-detect-aspects-checkpoint-local/tokenizer.json')

Prepare this model by adding an adapter to the original FLAN-T5 model. You are setting `is_trainable=False` because the plan is only to perform inference with this PEFT model. If you were preparing the model for further training, you would set `is_trainable=True`.

In [25]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       './peft-detect-aspects-checkpoint-local/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


The number of trainable parameters will be `0` due to `is_trainable=False` setting:

In [26]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 275889408
percentage of trainable model parameters: 0.00%


<a name=''></a>
###Evaluate the Model Qualitatively (Human Evaluation)

Make inferences for the same example as in above sections, with the original model, fully fine-tuned and PEFT model.

In [35]:
index = 1
review = my_iterable_dataset['test'][index]['review']
human_baseline_terms = my_iterable_dataset['test'][index]['terms']

prompt = f"""
Provide the comma separated aspects terms related to restuarant from below review.

{review}

Aspect Terms: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda')

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
peft_model.to('cuda')
peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
print(review)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_terms}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

Yet paired with such rude service, would never recommend for anyone interested in carrying any kind of conversation while there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
service
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
I'm not sure what's going on here.
---------------------------------------------------------------------------------------------------
PEFT MODEL: rude service


<a name=''></a>
###Evaluate the Model Quantitatively (with ROUGE Metric)
Perform inferences for the sample of the test dataset (only 100 dialogues and summaries to save time).

In [28]:
reviews = my_iterable_dataset['test'][:100]['review']
human_baseline_aspect_terms = my_iterable_dataset['test'][:100]['terms']

original_model_aspect_terms = []
peft_model_aspect_terms = []

for idx, review in enumerate(reviews):
    prompt = f"""
Provide the comma separated aspects terms related to restuarant from below review.


{review}

Aspect Terms: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda')

    human_baseline_text_output = human_baseline_aspect_terms[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    peft_model.to('cuda')
    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_aspect_terms.append(original_model_text_output)
    peft_model_aspect_terms.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_aspect_terms, original_model_aspect_terms, peft_model_aspect_terms))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_aspect_terms', 'original_model_aspect_terms',  'peft_model_aspect_terms'])
df.head()

,human_baseline_aspect_terms,original_model_aspect_terms,peft_model_aspect_terms
0,Thia food,:'s a's '''s a '' '''s '' '' '' '' '' '' '' ''...,restaurant
1,service,sucks,rude service
2,"dessert, White Chocolate Bread Pudding with Ge...",Dessert,dessert
3,"food, price","I'm not a fan of the food, but I'm not a fan o...",Good
4,"sushi, ceviche mix (special), crab dumplings, ...",sushi and rolls,sushi


In [29]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_aspect_terms,
    references=human_baseline_aspect_terms[0:len(original_model_aspect_terms)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_aspect_terms,
    references=human_baseline_aspect_terms[0:len(peft_model_aspect_terms)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)

print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.20624702715826615, 'rouge2': 0.0523465744400527, 'rougeL': 0.19983472417092485, 'rougeLsum': 0.20074521625537972}
PEFT MODEL:
{'rouge1': 0.4189729850709887, 'rouge2': 0.08419047619047618, 'rougeL': 0.41302702104811906, 'rougeLsum': 0.4157439466663604}


Notice, that PEFT model results are not too bad, while the training process was much easier!

In [30]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 21.27%
rouge2: 3.18%
rougeL: 21.32%
rougeLsum: 21.50%
